# Imports

In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import itertools
import random

# Check Gurobi license

In [2]:
try:

    # Create a new model
    m = gp.Model("2.17")

    # Create variables
    c = np.array([1, 2, 3])
    A = np.array([ [1, 2, 3], [1, 1, 0] ])
    b = np.array([4, 1])
    x_prev = np.array([1,2,3])
    x = m.addMVar(3, lb=0.0)
    m.setObjective(c @ x)
    m.addConstr(A@x == b)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-11
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x1990d0aa
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
C0 1
C1 0
C2 1
Obj: 4


# Functions

In [4]:
def min_Q_(c: dict, W: dict, h: dict, T: dict, G: dict, g: dict, x_prev: dict, k: int, t: int) -> tuple:

    """
    This function minimizes the function `c[t][k] @ x_t + phi` subject to the following constraints:

    * `W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev`
    * `- G[t+1] @ x_t + phi >= g[t+1]`

    Parameters TODO: Change back to dicts
    ----------
    c : numpy.ndarray
        A 2D array of cost coefficients.
    W : numpy.ndarray
        A 2D array of transition matrices.
    h : numpy.ndarray
        A 1D array of holding costs.
    T : numpy.ndarray
        A 2D array of transfer matrices.
    G : numpy.ndarray
        A 2D array of constraint matrices.
    g : numpy.ndarray
        A 1D array of constraint bounds.
    x_prev : numpy.ndarray
        A 1D array of previous state values.
    k : int
        A scenario index.
    t : int
        A time index.

    Returns
    -------
    x_t_val : numpy.ndarray
        A 1D array of optimal state values.
    obj_val : float
        The optimal objective value.
    pi : float
        The dual value of the first constraint.
    rho : float
        The dual value of the second constraint.
    """

    m = gp.Model("Q_")
    
    x_t = m.addMVar(shape = len(c[t][k]))#, lb=0.0)
    phi = m.addVar()
    
    m.setObjective(c[t][k] @ x_t + phi)
    
    sequence_constraint = m.addConstr(W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev, name = "sequence_constraint")
    cut_constraint = m.addConstr(- G[t+1] @ x_t + phi >= g[t+1] , name = "cut_constraint") #*np.ones_like(g[t+1])
    #print(G[t+1])
    #print(g[t+1])
    m.update()
    #print(f'RHS: {cut_constraint.getAttr("RHS")}')
        
    # Solve
    m.setParam( 'OutputFlag', False )
    m.optimize()
    
    # Primal and objective value
    x_t_val = x_t.getAttr("x")
    obj_val = m.getAttr("ObjVal")
    
    # Duals
    pi = sequence_constraint.getAttr("Pi")
    rho = cut_constraint.getAttr("Pi")

    # Return results
    return x_t_val, obj_val, pi, rho


In [5]:
def initialize_cut_matrices(c: dict, phi_: dict, t_max: int) -> tuple[dict]:
    assert len(c) == t_max
    G = {}
    g = {}

    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])

    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])

    return G, g

In [6]:
def calculate_new_cut(q: dict, p: dict, h: dict[int, dict], T: dict[int, dict], 
                      pi: dict[int, dict], rho: dict[int, dict], g: dict, 
                      i: int, t: int, scenario: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculates the new cut, i.e. beta and alpha based on the input dicts and dual variables.
    
    Args:
    - q: A dictionary representing the number of elements for each time period.
    - p: A dictionary representing the probabilities of choosing an element for each time period.
    - h: A dictionary of dictionaries representing the values for each element at each time period.
    - T: A dictionary of dictionaries representing the transition matrix for each element at each time period.
    - pi: A dictionary of dictionaries of dictionaries representing the transition probabilities of each element at each time period.
    - rho: A dictionary of dictionaries of dictionaries representing the values of each constraint at each time period.
    - g: A dictionary representing the objective function values for each time period.
    
    Returns:
    A tuple containing the values of beta and alpha.
    """
    beta = - sum(p[t][j] * np.matmul(pi[i][t][scenario][j],T[t-1][j]) for j in range(q[t]))
    alpha = sum(p[t][j] * (np.matmul(pi[i][t][scenario][j],h[t][j]) + np.dot(rho[i][t][scenario][j],g[t+1])) for j in range(q[t]))
    return beta, alpha

In [7]:
def add_cut(G_t: np.array, g_t: np.array, beta: np.array, alpha: np.array) -> tuple[np.array, np.array]:
    assert G_t.shape[0] == g_t.shape[0]
    
    G_t = np.vstack((G_t,beta))
    g_t = np.hstack((g_t,alpha))

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)
    G_t = hstacked_unique[:, :-1]
    g_t = hstacked_unique[:, -1]
    return G_t, g_t


In [8]:
def remove_cut(G_t: np.ndarray, g_t: np.ndarray, beta: np.ndarray, alpha: np.ndarray) -> tuple[np.ndarray, np.ndarray]:

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)

    plane = np.hstack((beta, alpha))
    hstacked_removed = hstacked_unique[(hstacked_unique != plane).any(axis=1)]

    G_t = hstacked_removed[:, :-1]
    g_t = hstacked_removed[:, -1]
    return G_t, g_t

In [17]:
def calculate_v_upper(c: dict,x_i: dict, sampled_scenarios: list, S: dict, t_max: int) -> tuple[float]:

    """
    Calculates the upper bound extimate for the value v and the standard deviation of the estimator
    """
    
    v_i = {scenario: sum( np.dot(c[t][S[scenario][t]], x_i[t][scenario]) for t in range(1, t_max+1) ) for scenario in sampled_scenarios}
    v_upper_i = sum( v_i[scenario] for scenario in sampled_scenarios )
    v_upper_i = v_upper_i / len(sampled_scenarios)

    sigma_v_i = sum( np.square(v_i[scenario] - v_upper_i) for scenario in sampled_scenarios )
    sigma_v_i = sigma_v_i / ( (len(sampled_scenarios)-1) * len(sampled_scenarios) ) # Added to have standardisation
    sigma_v_i = np.sqrt(sigma_v_i)

    return v_upper_i, sigma_v_i


# SDDP

In [10]:
def SDDP(t_max: int, i_max: int, p: dict, q: dict, S: dict, c: dict, W: dict, h: dict, T: dict, phi_: np.array, no_of_samples: int, x_0: np.array) -> tuple:
    i = 0
    x = {}
    Q_ = {}
    pi = {}
    rho = {}
  
    v_lower = {}
    v_upper = {}
    sigma_v_upper = {}  

    # Initialize dicts
    for i in range(1, i_max+1):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            for scenario in list(S.keys()):
                pi[i][t][scenario] = {}
                rho[i][t][scenario] = {}

    print(f'dicts initialized, {len(list(S.keys()))} scenarios')
    
    
    # Initialize cut matrices
    G, g = initialize_cut_matrices(c=c, phi_=phi_, t_max=t_max)
    print('cut approximations initialized')
    
    
    i = 1

    while i <= i_max:
        
        ## FORWARD PASS
        print(f"Forward pass: {i}")
        sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
        for t in range(1,t_max+1):
            for scenario in sampled_scenarios:
                #print(f"t: {t}, scenario: {scenario}")
                k = S[scenario][t]
                x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t) # Seems good
        
        # Calculate upper bound estimate for v
        v_upper[i], sigma_v_upper[i] = calculate_v_upper(c, x[i], sampled_scenarios, S, t_max)
        print(f"v upper {i}: {v_upper[i]}, sigma v upper {i}: {sigma_v_upper[i]}")
    
        
        ## BACKWARD PASS
        for t in reversed(range(2,t_max+1)):
            for scenario in sampled_scenarios:
                
                # Compute duals (pi, rho) for the scenario
                for j in range(q[t]):
                    x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                
                if t != 1:
                    # Calculate new cut (beta, alpha)
                    beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                    # Add new cut (beta, alpha) to G, g:
                
                    G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                    
        # Finally solve stage t = 1 to get v_lower
        t = 1
        x[i][t][0], Q_[i][t][0], pi_temp, rho_temp = min_Q_(c,W,h,T,G,g,x[i][t-1][0],0,t)

        v_lower[i] = Q_[i][t][0]
        print(f"v lower {i}: {v_lower[i]}")
        i = i + 1
        
    return x, Q_, v_lower, v_upper

# BL-SDDP

In [11]:
def BL_SDDP(t_max: int, i_max: int, z_max: int, p: dict, q: dict, S: dict[int, tuple[int]], c: dict, W: dict, h: dict, T: dict, 
            phi_: np.array, no_of_samples: int, batch_size: int, x_0: np.array) -> tuple:
    
    assert batch_size <= no_of_samples, "Batch size larger than number of samples per stage"


    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    v_lower = {}
    v_upper = {}
    sigma_v_upper = {}

    memory = {}
    
    # Initialize cut matrices
    G, g = initialize_cut_matrices(c=c, phi_=phi_, t_max=t_max)
    print('cut approximations initialized')


     # Initialize dicts
    for i in range(1, i_max+1):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            for scenario in list(S.keys()):
                pi[i][t][scenario] = {}
                rho[i][t][scenario] = {}
            

        for t in range(1,t_max+1):
            memory[t] = {}

    print(f'dicts initialized, {len(list(S.keys()))} scenarios')
    

    i = 1

    while i <= i_max:
        print(f'Iteration {i}:')

        z = 0
        while z < z_max:


            # Sample no_of_samples scenarios
            sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
            print(f"Sampled scenarios: {sampled_scenarios}")
            
            ## FORWARD PASS

            # Compute actions (forward)
            for t in range(1,t_max+1):
                for scenario in sampled_scenarios:
                    
                    k = S[scenario][t]
                    x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t)
        
                    # Add trial action to memory
                    memory[t][(i,scenario)] = {"action": x[i][t][scenario]}


            # Calculate upper bound estimate for v
            v_upper[i], sigma_v_upper[i] = calculate_v_upper(c, x[i], sampled_scenarios, S, t_max)
            print(f"v upper {i}: {v_upper[i]}, sigma v upper {i}: {sigma_v_upper[i]}")


            ## BACKWARD PASS
            
            for t in reversed(range(2,t_max+1)):
                for scenario in sampled_scenarios:
                    
                    # Compute duals (pi, rho) for the scenario
                    for j in range(q[t]):
                    
                        x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                    
                    if t != 1:
                        # Calculate new cut (beta, alpha)
                        beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)
  
                        # Add new cut (beta, alpha) to G, g:
                    
                        G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                        # Store cut
                        memory[t][(i,scenario)]["beta"] = beta
                        memory[t][(i,scenario)]["alpha"] = alpha

            # Increase z
            z = z + no_of_samples
            
        # Select batch keys
        batch_keys = {}
        for t in range(2, t_max+1):

            # Select batch_size number of keys from the memory for stage t
            batch_keys[t] = random.sample(list(memory[t]), batch_size)

            # Remove the K cuts belonging to the K selected actions
            for batch_key in batch_keys[t]:
                beta = memory[t][batch_key]["beta"]
                alpha = memory[t][batch_key]["alpha"]
                if t != 1:
                    G[t], g[t] = remove_cut(G[t], g[t], beta, alpha)
        

        # Backward pass around the K trial actions
        for t in reversed(range(2,t_max+1)):
            for batch_key in batch_keys[t]:
                
                # Get scenario and iteration of the original
                i_batch, scenario = batch_key

                # Compute duals (pi, rho) for the scenario
                for j in range(q[t]):
                    # print((t, batch_key, i, i_batch))
                    # print(x[i_batch][t-1][scenario])
                    x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i_batch][t-1][scenario],j,t)
                
                # Calculate new cut (beta, alpha)
                beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                # Add new cut (beta, alpha) to G, g:
                if t != 1:
                    G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                # Store new cut in memory (exactly where the removed cut was taken from)
                memory[t][batch_key]["beta"] = beta
                memory[t][batch_key]["alpha"] = alpha

        # Finally solve stage t = 1 to get v_lower
        t = 1
        x[i][t][0], Q_[i][t][0], pi_temp, rho_temp = min_Q_(c,W,h,T,G,g,x[i][t-1][0],0,t)

        v_lower[i] = Q_[i][t][0]
        print(f"v lower {i}: {v_lower[i]}")
            
        i = i + 1

    return x, Q_, v_lower, v_upper





# Examples

## Data functions

In [13]:
def generate_scenarios(q: dict, t_max: int) -> dict:
    """
    Generates the scenario set S based on the stage uncertainties q and t_max
    """
    assert len(q) == t_max
    
    list_q = [range(q[t]) for t in range(1,t_max+1)]
    scenarios_raw = [w for w in itertools.product(*list_q)]
    S = {i: {t: scenarios_raw[i][t-1] for t in range(1,len(scenarios_raw[i])+1)} for i in range(len(scenarios_raw))}
    return S

## Toy example

In [114]:
## INPUT
t_max = 3

# Uncertainty
q = {t: 3 for t in range(2,t_max+1)}
q[1] = 1
print(q)

p = {t: np.array([0.3333,0.3334,0.3333]) for t in range(2,t_max+1)}

S = generate_scenarios(q=q, t_max=t_max)
print(S)


# Initial condition
x_0 = np.array([ 0, 0 ])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}

for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}

c[1][0] = np.array([ 1, 0 ])
c[2][0] = np.array([ 1, 0 ])
c[2][1] = np.array([ 1, 0 ])
c[2][2] = np.array([ 1, 0 ])
c[3][0] = np.array([ 1, 1 ])
c[3][1] = np.array([ 1, 1 ])
c[3][2] = np.array([ 1, 1 ])

W[1][0] = np.array([ [1, 1] ])
W[2][0] = np.array([ [1, -1] ])
W[2][1] = np.array([ [1, -1] ])
W[2][2] = np.array([ [1, -1] ])
W[3][0] = np.array([ [1, -1] ])
W[3][1] = np.array([ [1, -1] ])
W[3][2] = np.array([ [1, -1] ])

h[1][0] = np.array([ 6 ])
h[2][0] = np.array([ 4 ])
h[2][1] = np.array([ 5 ])
h[2][2] = np.array([ 6 ])
h[3][0] = np.array([ 1 ])
h[3][1] = np.array([ 2 ])
h[3][2] = np.array([ 4 ])

T[0][0] = np.array([ [0, 0] ])
T[1][0] = np.array([ [1, 0] ])
T[1][1] = np.array([ [1, 0] ])
T[1][2] = np.array([ [1, 0] ])
T[2][0] = np.array([ [1, 0] ])
T[2][1] = np.array([ [1, 0] ])
T[2][2] = np.array([ [1, 0] ])


# Algorithm parameters
i_max = 40
z_max = 1
no_of_samples = 9
batch_size = 1
phi_list = [-100 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])
print(phi_)


{2: 3, 3: 3, 1: 1}
{0: {1: 0, 2: 0, 3: 0}, 1: {1: 0, 2: 0, 3: 1}, 2: {1: 0, 2: 0, 3: 2}, 3: {1: 0, 2: 1, 3: 0}, 4: {1: 0, 2: 1, 3: 1}, 5: {1: 0, 2: 1, 3: 2}, 6: {1: 0, 2: 2, 3: 0}, 7: {1: 0, 2: 2, 3: 1}, 8: {1: 0, 2: 2, 3: 2}}
[   0 -100 -100 -100]


In [115]:
x, Q_, v_lower, v_upper = SDDP(t_max=t_max, i_max=i_max,
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples,
                                  x_0 = x_0)

dicts initialized, 9 scenarios
cut approximations initialized
Forward pass: 1
v upper 1: 7.666666666666667, sigma v upper 1: 0.7264831572567789
v lower 1: 4.312410945116886
Forward pass: 2
v upper 2: 6.798541846202024, sigma v upper 2: 0.4084838045398825
v lower 2: 5.5554444499999995
Forward pass: 3
v upper 3: 6.253964852734029, sigma v upper 3: 0.42510322996606476
v lower 3: 6.166566661250375
Forward pass: 4
v upper 4: 6.388888885139263, sigma v upper 4: 0.3888888891567193
v lower 4: 6.222144439999999
Forward pass: 5
v upper 5: 6.222222222222222, sigma v upper 5: 0.43390275977259196
v lower 5: 6.222144439999999
Forward pass: 6
v upper 6: 6.222222222222222, sigma v upper 6: 0.43390275977259196
v lower 6: 6.222144439999999
Forward pass: 7
v upper 7: 6.222222222222222, sigma v upper 7: 0.4339027597725919
v lower 7: 6.222144439999999
Forward pass: 8
v upper 8: 6.222222222222222, sigma v upper 8: 0.43390275977259196
v lower 8: 6.222144439999999
Forward pass: 9
v upper 9: 6.222222222222222,

In [121]:
x[40][1]

{7: array([3., 3.]),
 1: array([3., 3.]),
 6: array([3., 3.]),
 8: array([3., 3.]),
 2: array([3., 3.]),
 4: array([3., 3.]),
 5: array([3., 3.]),
 0: array([3., 3.]),
 3: array([3., 3.])}

In [113]:
x, Q_, v_lower, v_upper = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 9 scenarios
Iteration 1:
Sampled scenarios: [5, 2, 6, 1, 8, 0]
v upper 1: 7.0, sigma v upper 1: 0.9660917830792959
v lower 1: 4.312410945116886
Iteration 2:
Sampled scenarios: [6, 7, 8, 0, 3, 4]
v upper 2: 6.374940630077925, sigma v upper 2: 0.451563906331459
v lower 2: 5.5554444499999995
Iteration 3:
Sampled scenarios: [4, 3, 6, 1, 0, 8]
v upper 3: 5.928561224868754, sigma v upper 3: 0.6024312583427583
v lower 3: 6.166566661250375
Iteration 4:
Sampled scenarios: [0, 7, 5, 4, 2, 1]
v upper 4: 6.166666659167416, sigma v upper 4: 0.49441323297864354
v lower 4: 6.222144439999999
Iteration 5:
Sampled scenarios: [0, 7, 2, 5, 3, 8]
v upper 5: 6.333333333333333, sigma v upper 5: 0.49441323247304414
v lower 5: 6.222144439999999
Iteration 6:
Sampled scenarios: [3, 7, 1, 8, 0, 5]
v upper 6: 6.0, sigma v upper 6: 0.5163977794943222
v lower 6: 6.222144439999999
Iteration 7:
Sampled scenarios: [7, 4, 3, 8, 6, 0]
v upper 7: 6.166666666666667, sigma v

In [38]:
x[39][1]

{5: array([3., 3.]),
 7: array([3., 3.]),
 2: array([3., 3.]),
 8: array([3., 3.]),
 4: array([3., 3.]),
 0: array([3., 3.])}

## Energy example

In [19]:
## INPUT

t_max = 12

# Uncertainty
q = {t: 2 for t in range(2,t_max+1)}
q[1] = 1

p = {t: np.array([0.45,0.55]) for t in range(2,t_max+1)}

S = generate_scenarios(q=q, t_max=t_max)
print(f"No. of scenarios: {len(S)}")

# Parameters
from_grid_max = 25
to_grid_max = 25
soc_max = 100

price_from_grid = {t: {} for t in range(1,t_max+1)}
price_to_grid = {t: {} for t in range(1,t_max+1)}
gen_less_demand = {t: {} for t in range(1,t_max+1)}

for t in range(1, 2):
    price_from_grid[t] = {0: 0.3}
    price_to_grid[t] = {0: 0.15}
    gen_less_demand[t] = {0: 0}

for t in range(2, 7):
    price_from_grid[t] = {0: 0.2, 1: 0.4}
    price_to_grid[t] = {0: 0.1, 1: 0.2} 
    gen_less_demand[t] = {0: 0, 1: 10}

for t in range(7, t_max+1):
    price_from_grid[t] = {0: 0.4, 1: 0.6}
    price_to_grid[t] = {0: 0.2, 1: 0.4}
    gen_less_demand[t] = {0: -20, 1: -5}

# Initial condition
x_0 = np.array([0,0,10,0,0,0])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}
for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}
    for j in range(q[t]):
        c[t][j] = np.array([price_from_grid[t][j], -price_to_grid[t][j], 0, 0, 0, 0])
        W[t][j] = np.array([ [-1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1] ])
        h[t][j] = np.array([gen_less_demand[t][j], from_grid_max, to_grid_max, soc_max])
        T[t-1][j] = np.array([ [0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0] ])


No. of scenarios: 2048


In [20]:
# Algorithm parameters
i_max = 50
z_max = 10
no_of_samples = 60
batch_size = 30
phi_list = [-20 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])

In [21]:
result_sddp = {}
result_sddp["x"], result_sddp["Q_"], result_sddp["v_lower"], result_sddp["v_upper"] = SDDP(t_max=t_max, i_max=i_max,
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples,
                                  x_0 = x_0)

dicts initialized, 2048 scenarios
cut approximations initialized
Forward pass: 1
v upper 1: 26.566666666666666, sigma v upper 1: 0.9817160884607877
v lower 1: -1.5
Forward pass: 2
v upper 2: 15.976683847947536, sigma v upper 2: 0.9967840952914804
v lower 2: 1.4625000000000101
Forward pass: 3
v upper 3: 10.457916666666666, sigma v upper 3: 0.8042587236176344
v lower 3: 6.198844948152125
Forward pass: 4
v upper 4: 9.303333333333324, sigma v upper 4: 1.0076346040564022
v lower 4: 6.705689718750007
Forward pass: 5
v upper 5: 10.073333333333322, sigma v upper 5: 1.013111408942926
v lower 5: 6.705689718750007
Forward pass: 6
v upper 6: 9.69999999999999, sigma v upper 6: 1.006981280578258
v lower 6: 6.705689718750007
Forward pass: 7
v upper 7: 8.33499999999999, sigma v upper 7: 0.9833438427093294
v lower 7: 6.705689718750007
Forward pass: 8
v upper 8: 7.739999999999992, sigma v upper 8: 0.8667270730795478
v lower 8: 6.705689718750007
Forward pass: 9
v upper 9: 10.066666666666656, sigma v uppe

In [37]:
result_sddp["x"][50][12][150]

array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ])

In [22]:
result_blsddp = {}
result_blsddp["x"], result_blsddp["Q_"], result_blsddp["v_lower"], result_blsddp["v_upper"] = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  p=p, q=q, S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 2048 scenarios
Iteration 1:
Sampled scenarios: [616, 1688, 1302, 1465, 1612, 1958, 1371, 1985, 1522, 1475, 960, 899, 1883, 1404, 452, 654, 633, 1877, 1453, 1743, 1385, 638, 608, 636, 1139, 1515, 294, 1466, 280, 1935, 1853, 1417, 1648, 562, 1808, 368, 704, 1786, 1768, 1579, 1622, 423, 1299, 457, 44, 1912, 1017, 596, 434, 1446, 1192, 1055, 2011, 1901, 381, 820, 1265, 1459, 1570, 768]
v upper 1: 26.233333333333334, sigma v upper 1: 0.8423355567748261
v lower 1: -1.5
Iteration 2:
Sampled scenarios: [1522, 485, 875, 1848, 1435, 6, 1140, 1604, 1874, 1998, 1650, 943, 953, 1835, 1194, 1534, 1031, 477, 1071, 2043, 272, 1316, 391, 571, 628, 195, 1894, 1516, 1392, 1389, 1393, 1241, 1450, 1530, 665, 538, 1624, 192, 1710, 753, 727, 925, 851, 0, 577, 1078, 63, 341, 1617, 1573, 664, 67, 1349, 1449, 278, 499, 1942, 1843, 1793, 285]
v upper 2: 14.332248565368001, sigma v upper 2: 1.0174300093265105
v lower 2: 1.4625000000000101
Iteration 3:
Sampled scen

In [38]:
result_blsddp["x"][50][12]

{1968: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 1273: array([  0.  ,   6.75,   0.  ,  25.  ,  18.25, 100.  ]),
 306: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 607: array([  0.  ,   6.75,   0.  ,  25.  ,  18.25, 100.  ]),
 878: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 1598: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 430: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 577: array([  0.  ,   6.75,   0.  ,  25.  ,  18.25, 100.  ]),
 1140: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 874: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 466: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 1506: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 1798: array([  8.25,   0.  ,   0.  ,  16.75,  25.  , 100.  ]),
 2009: array([  0.  ,   6.75,   0.  ,  25.  ,  18.25, 100.  ]),
 283: array([  0.  ,   6.75,   0.  ,  25.  ,  18.25, 100.  ]),
 586: array([  8.25,   0.  ,   0.  ,  16.75,  25